In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
from utils.spark_config import create_spark_session
from src.data_loader import DataLoader
from src.eda_analyser import EdaAnalyser
from src.preprocessor import Preprocessor


In [3]:
spark = create_spark_session()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/24 15:18:54 WARN Utils: Your hostname, Jakubs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.157 instead (on interface en0)
25/12/24 15:18:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/24 15:18:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data_loader = DataLoader(spark)
df = data_loader.load_file()

In [5]:
df = Preprocessor.run(df)

In [6]:
df.limit(5).show()

+--------------------+----+--------------------+----+-----+-----------------------+--------------------+-----+-------------+-----------+--------+--------------------+
|               title| tag|              artist|year|views|               features|              lyrics|   id|language_cld3|language_ft|language|      lyrics_cleaned|
+--------------------+----+--------------------+----+-----+-----------------------+--------------------+-----+-------------+-----------+--------+--------------------+
|           Freestyle| rap|              Flamez|2011|   10|                  {Non}|The story of the ...| 4291|           en|         en|      en|The story of the ...|
|                 HMU| rap|         (Moonshine)|2011|   84|"{""문샤인 (Moonshin...|I'll be a shootin...| 5692|           en|         en|      en|I'll be a shootin...|
|The U.S. Constitu...|misc|Constitutional Co...|1788|10877|   "{""James Madison...|We the People of ...| 6201|           en|         en|      en|We the People of ...|
|

In [ ]:
analyser = EdaAnalyser(df)
analyser.run_full_eda_report(['title', 'lyrics', 'views'])

Columns and their datatypes present in the dataset:
root
 |-- title: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- views: integer (nullable = true)
 |-- features: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- language_cld3: string (nullable = true)
 |-- language_ft: string (nullable = true)
 |-- language: string (nullable = true)
 |-- lyrics_cleaned: string (nullable = true)

Sample 5 rows:
+--------------------+----+--------------------+----+-----+-----------------------+--------------------+-----+-------------+-----------+--------+--------------------+
|               title| tag|              artist|year|views|               features|              lyrics|   id|language_cld3|language_ft|language|      lyrics_cleaned|
+--------------------+----+--------------------+----+-----+-----------------------+--------------------+-----+-----------